In [1]:
import requests 

import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from datetime import datetime
from dotenv import load_dotenv              # environment variables
import os

In [2]:
# load the environment variables
load_dotenv()

# assign environment variables
PASSWORD = os.getenv('MariaDB_Password')

In [3]:
spark = SparkSession.builder.appName("Credit Card App").getOrCreate()

External API

In [4]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'

Extract Function

In [16]:
# extract api
def extract_api(url):
    data = requests.get(url)
    print('Status code:', data.status_code)
    
    response = data.json()
    return response 

In [18]:
loan_data = extract_api(url)
loan_data[0]

Status code: 200


{'Application_ID': 'LP001002',
 'Gender': 'Male',
 'Married': 'No',
 'Dependents': '0',
 'Education': 'Graduate',
 'Self_Employed': 'No',
 'Credit_History': 1,
 'Property_Area': 'Urban',
 'Income': 'medium',
 'Application_Status': 'Y'}